In [1]:
from models.model import ViTBase
from models.utils import train_step, eval_step, DataLoaders
from models.transforms import transforms_resnet
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10

In [2]:
train = CIFAR10(root='data',transform=transforms_resnet,download=True)
test = CIFAR10(root='data',transform=transforms_resnet,train=False,download=True)

Files already downloaded and verified
Files already downloaded and verified


In [19]:
BATCH_SIZE = 64
N_EPOCHS = 2

In [20]:
dl = DataLoaders(train,test,'resnet',BATCH_SIZE,True,'cifar10')

In [21]:
train_loader, test_loader = dl.get_loaders()

In [22]:
# Train and then Evaluate (Three Different Train and Evaluation Loops)
from tqdm import tqdm
def train_and_eval(train_loader,test_loader,model,loss_fn,optimizer,device,modeltype):
    tr_metric = {"Accuracy":[],"Loss":[]}
    ts_metric = {"Accuracy":[],"Loss":[]}

    for epoch in tqdm(range(N_EPOCHS)):
        tr_loss, tr_acc = train_step(model,train_loader,loss_fn,optimizer,device,modeltype)
        ts_loss, ts_acc = eval_step(model,test_loader,loss_fn,device,modeltype,data="cifar10")

        tr_metric["Accuracy"].append(tr_acc)
        tr_metric["Loss"].append(tr_loss)

        ts_metric["Accuracy"].append(ts_acc)
        ts_metric["Loss"].append(ts_loss)
    
    return tr_metric, ts_metric

model = ViTBase(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,model,loss_func,optimizer,device,'vitbase')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])

100%|██████████| 2/2 [05:36<00:00, 168.13s/it]

Final Train Accuracy: 0.9449
Final Test Accuracy: 0.94449997
Final Train Loss: 0.16424956239636063
Final Test Loss: 0.1746053358385707


In [25]:
import pandas as pd
pd.DataFrame(tr).to_csv("vitbase_cifar10_tr_1.csv") 
pd.DataFrame(ts).to_csv("vitbase_cifar10_ts_1.csv")

In [26]:
torch.save(model.state_dict(),"vitbase_cifar10.pth")
model.load_state_dict(torch.load("vitbase_cifar10.pth"))   

C:\Users\PCF\AppData\Local\Temp\ipykernel_18836\1203643036.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("vitbase_cifar10.pth"))


<All keys matched successfully>